In [14]:
import torch
import numpy as np
from torch import nn

In [3]:
layer1 = nn.Linear(8,3)

In [4]:
torch.ones((3,8))

tensor([[1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [5]:
layer1.weight.size(1)

8

In [6]:
layer1.weight[:,4:], layer1.weight[:,:4]

(tensor([[-0.0581,  0.0361, -0.0649,  0.0933],
         [ 0.2731, -0.2574, -0.2784, -0.2805],
         [ 0.2419,  0.2746,  0.2415,  0.3205]], grad_fn=<SliceBackward0>),
 tensor([[-0.3356, -0.2775,  0.2027,  0.0922],
         [-0.1183,  0.1392, -0.2094,  0.2146],
         [-0.0479,  0.0785,  0.1698, -0.1927]], grad_fn=<SliceBackward0>))

In [7]:
x = torch.ones((2,3))
y = torch.zeros((2,6))
z = torch.cat((x,y),dim=1)
z

tensor([[1., 1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0., 0.]])

In [8]:
x = torch.ones((18,20))
gama, beta = torch.split(x,10,1)
gama.shape, beta.shape

(torch.Size([18, 10]), torch.Size([18, 10]))

In [26]:
from transformers import BertModel, BertConfig, BertTokenizer
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType
from peft.utils.config import PromptLearningConfig

bertconfig = BertConfig.from_pretrained('bert-base-uncased', output_hidden_states=True)

peft_config = LoraConfig(
    task_type=TaskType.TOKEN_CLS, inference_mode=True, r=8, lora_alpha=32, lora_dropout=0.1
)
# print(isinstance(peft_config, PromptLearningConfig)) # False

model = BertModel.from_pretrained('bert-base-uncased', config=bertconfig)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()
# trainable params: 294912 || all params: 109777152 || trainable%: 0.2686460658042941

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


trainable params: 294912 || all params: 109777152 || trainable%: 0.2686460658042941


In [27]:
import torch
sentence1 = "I love the sunny day"
sentence2 = "I love my mom dad"
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',padding=True,truncation=True,max_length=512,return_tensors="pt")
input1 = tokenizer.encode_plus(sentence1)
input2 = tokenizer.encode_plus(sentence2)
print(input1)
input_ids=torch.concat((torch.unsqueeze(torch.LongTensor(input1['input_ids']),dim=0),torch.unsqueeze(torch.LongTensor(input2['input_ids']),dim=0)),dim=0)
print(input_ids.shape)

{'input_ids': [101, 1045, 2293, 1996, 11559, 2154, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}
torch.Size([2, 7])


In [28]:
origin_model = BertModel.from_pretrained('bert-base-uncased', config=bertconfig)
x = origin_model(input_ids=input_ids)
print(x)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 0.1219,  0.2386, -0.0089,  ..., -0.3565,  0.2120,  0.2568],
         [ 0.3584,  0.1863, -0.2557,  ..., -0.4161,  0.5436,  0.0312],
         [ 1.2115,  0.7444,  1.1416,  ..., -0.1788,  0.4152, -0.1766],
         ...,
         [-1.3715, -0.0884, -0.2859,  ...,  0.5654, -0.0344, -0.1637],
         [-0.4437, -0.2219, -0.1582,  ...,  0.6177,  0.3368, -0.4463],
         [ 0.6994, -0.0115, -0.0842,  ..., -0.1018, -0.4322, -0.3810]],

        [[ 0.0987,  0.3710, -0.1172,  ..., -0.2921,  0.2551,  0.1856],
         [ 0.1952,  0.3604, -0.4829,  ...,  0.1172,  0.9087,  0.2542],
         [ 0.9428,  0.7529,  0.6970,  ..., -0.5273,  0.7027,  0.0036],
         ...,
         [ 0.4026, -0.0351,  0.1042,  ..., -0.3336,  0.6649,  0.0607],
         [-0.0187, -0.4869, -0.1939,  ...,  0.6178,  0.6148, -0.1918],
         [ 0.6716,  0.1270, -0.1164,  ..., -0.2789, -0.5492, -0.3206]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_ou

In [29]:
y = model.base_model(input_ids=input_ids)
print(y)

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 0.1219,  0.2386, -0.0089,  ..., -0.3565,  0.2120,  0.2568],
         [ 0.3584,  0.1863, -0.2557,  ..., -0.4161,  0.5436,  0.0312],
         [ 1.2115,  0.7444,  1.1416,  ..., -0.1788,  0.4152, -0.1766],
         ...,
         [-1.3715, -0.0884, -0.2859,  ...,  0.5654, -0.0344, -0.1637],
         [-0.4437, -0.2219, -0.1582,  ...,  0.6177,  0.3368, -0.4463],
         [ 0.6994, -0.0115, -0.0842,  ..., -0.1018, -0.4322, -0.3810]],

        [[ 0.0987,  0.3710, -0.1172,  ..., -0.2921,  0.2551,  0.1856],
         [ 0.1952,  0.3604, -0.4829,  ...,  0.1172,  0.9087,  0.2542],
         [ 0.9428,  0.7529,  0.6970,  ..., -0.5273,  0.7027,  0.0036],
         ...,
         [ 0.4026, -0.0351,  0.1042,  ..., -0.3336,  0.6649,  0.0607],
         [-0.0187, -0.4869, -0.1939,  ...,  0.6178,  0.6148, -0.1918],
         [ 0.6716,  0.1270, -0.1164,  ..., -0.2789, -0.5492, -0.3206]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_ou

In [30]:
print(sum(param.numel() for param in origin_model.parameters())) # 109482240
print(sum(param.numel() for param in model.base_model.parameters())) # 109777152
# print(sum(param.numel() for param in model.base_model.parameters()))

109482240
109777152


In [31]:
x = 0
for params in model.base_model.parameters():
    if params.requires_grad == True:
        x += params.numel()

print(x) # 294912

294912


In [16]:
model(input_ids=input['input_ids'],attention_mask=input['attention_mask'],
      labels=label)

TypeError: forward() got an unexpected keyword argument 'labels'

In [14]:
# TARGET_MODULES = [
#     "query",
#     "v_proj",
# ]

# TARGET_MODULES = "v_proj"

In [17]:
import re
key_list = [key for key, _ in bertModel.named_modules()]
for i in list(range(len(key_list))):
     # target_module_found = any(key_list[i].endswith(target_key) for target_key in TARGET_MODULES)
     # if target_module_found:
    print(key_list[i])


embeddings
embeddings.word_embeddings
embeddings.position_embeddings
embeddings.token_type_embeddings
embeddings.LayerNorm
embeddings.dropout
encoder
encoder.layer
encoder.layer.0
encoder.layer.0.attention
encoder.layer.0.attention.self
encoder.layer.0.attention.self.query
encoder.layer.0.attention.self.key
encoder.layer.0.attention.self.value
encoder.layer.0.attention.self.dropout
encoder.layer.0.attention.output
encoder.layer.0.attention.output.dense
encoder.layer.0.attention.output.LayerNorm
encoder.layer.0.attention.output.dropout
encoder.layer.0.intermediate
encoder.layer.0.intermediate.dense
encoder.layer.0.intermediate.intermediate_act_fn
encoder.layer.0.output
encoder.layer.0.output.dense
encoder.layer.0.output.LayerNorm
encoder.layer.0.output.dropout
encoder.layer.1
encoder.layer.1.attention
encoder.layer.1.attention.self
encoder.layer.1.attention.self.query
encoder.layer.1.attention.self.key
encoder.layer.1.attention.self.value
encoder.layer.1.attention.self.dropout
encoder.

In [43]:
import loralib as lora
# Add a pair of low-rank adaptation matrices with rank r=16
layer = lora.Linear(512, 512, r=16)
lora.mark_only_lora_as_trainable(bertModel)

In [33]:
import torch
print(torch.sub(input=y[1],other=x[1]))

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.

In [9]:
from transformers import BertModel, BertConfig, BertTokenizer
from peft import get_peft_model, LoraConfig, TaskType

bertconfig = BertConfig.from_pretrained('bert-base-uncased', output_hidden_states=True)
model = BertModel.from_pretrained('bert-base-uncased', config=bertconfig)
peft_config = LoraConfig(
    task_type=TaskType.TOKEN_CLS, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1
)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


trainable params: 294912 || all params: 109777152 || trainable%: 0.2686460658042941


In [2]:
import loralib as lora

# lora.mark_only_lora_as_trainable(model)

In [8]:
for n, p in model.named_parameters():
    print(n)
    print(p.shape)

embeddings.word_embeddings.weight
torch.Size([30522, 768])
embeddings.position_embeddings.weight
torch.Size([512, 768])
embeddings.token_type_embeddings.weight
torch.Size([2, 768])
embeddings.LayerNorm.weight
torch.Size([768])
embeddings.LayerNorm.bias
torch.Size([768])
encoder.layer.0.attention.self.query.weight
torch.Size([768, 768])
encoder.layer.0.attention.self.query.bias
torch.Size([768])
encoder.layer.0.attention.self.key.weight
torch.Size([768, 768])
encoder.layer.0.attention.self.key.bias
torch.Size([768])
encoder.layer.0.attention.self.value.weight
torch.Size([768, 768])
encoder.layer.0.attention.self.value.bias
torch.Size([768])
encoder.layer.0.attention.output.dense.weight
torch.Size([768, 768])
encoder.layer.0.attention.output.dense.bias
torch.Size([768])
encoder.layer.0.attention.output.LayerNorm.weight
torch.Size([768])
encoder.layer.0.attention.output.LayerNorm.bias
torch.Size([768])
encoder.layer.0.intermediate.dense.weight
torch.Size([3072, 768])
encoder.layer.0.inter

In [4]:
from modules.attention.transformer import TransformerEncoder
import torch
enc = TransformerEncoder(embed_dim=32, num_heads=8, layers=4, attn_mask=False)
audio = torch.ones((32, 289, 74))
layer = torch.nn.Linear(74, 32)
audio = layer(audio)
x = enc(audio)
print(x.shape)

torch.Size([32, 289, 32])


In [3]:
print(torch.Tensor(2, 2, 3))

tensor([[[0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.]]])


In [6]:
import torch
n = torch.ones((32, 124, 1))
print(torch.sum(n))

tensor(3968.)


In [7]:
from modules.fusion import BottleNeckFusion
from config import Config
import torch
config = Config()
fusion = BottleNeckFusion(config)
audio = torch.ones((32, 194, 74))
visual = torch.ones((32, 268, 35))
text = torch.ones((32, 50, 768))
a, v, t, result = fusion(audio,visual,text)

In [14]:
print(a)
print(v)
print(t)

tensor(3577.6467, grad_fn=<AddBackward0>)
tensor(2652.1299, grad_fn=<AddBackward0>)
tensor(176.9038, grad_fn=<AddBackward0>)


In [31]:
dict_sc = {'audio':a,'visual':v,'text':t}
sorted(dict_sc)
dict_sc

{'audio': tensor(3577.6467, grad_fn=<AddBackward0>),
 'visual': tensor(2652.1299, grad_fn=<AddBackward0>),
 'text': tensor(176.9038, grad_fn=<AddBackward0>)}

In [33]:
for index, key in enumerate(dict_sc):
    print(index)
    print(key)
    print(dict_sc[key])

0
audio
tensor(3577.6467, grad_fn=<AddBackward0>)
1
visual
tensor(2652.1299, grad_fn=<AddBackward0>)
2
text
tensor(176.9038, grad_fn=<AddBackward0>)


In [30]:
print(dict_sc[])

[tensor(3577.6467, grad_fn=<AddBackward0>), tensor(2652.1299, grad_fn=<AddBackward0>), tensor(176.9038, grad_fn=<AddBackward0>)]


In [11]:
1 - torch.tanh(0.3 * torch.relu(torch.tensor([3])))

tensor([0.2837])

In [12]:
1 - torch.tanh(0.3 * torch.tensor([3]))

tensor([0.2837])

In [13]:
1 - torch.tanh(0.3 * torch.tensor([5]))

tensor([0.0949])

In [34]:
x = torch.tensor([[1,2,3],[3,4,5]])
x[:,1:3]

tensor([[2, 3],
        [4, 5]])